In [2]:
import pandas as pd
from TableOne import TableOne

ModuleNotFoundError: No module named 'TableOne'

# New Onset Dosing in T1D

In [ ]:
# Data import and cleaning
wd = '/Volumes/PEDS/RI Biostatistics Core/Shared/Shared Projects/Laura/BDC/Projects/Kimber Simmons/New Onset Dosing/Data_Cleaned/'
df = pd.read_csv(wd+'new_onset_doses.csv')
df.set_index(['MRN'],inplace=True)
df

## Table 1: Participant Characteristics at Visit 1

In [2]:
# Format columns
# Convert all ages to months
df['Age'] = [int(a.split()[0])*12 if 'year' in a else int(a.split()[0]) for a in df['Age']]
# Make table 1
cols = ['Age']

# Print
t1 = TableOne(df,columns=cols,groupby='Sex',pval=True,display_all=True,
             normal_test=True,tukey_test=True,dip_test=True)
t1

NameError: name 'df' is not defined

In [23]:
s = '2 year old'
s.split()

MRN
2240959.0     2 year old
1948268.0     3 year old
1600154.0     6 year old
1395523.0     8 year old
2153166.0     4 year old
                ...     
1581860.0     7 year old
1883313.0     3 year old
2218098.0     9 year old
1370356.0    13 year old
986173.0     15 year old
Name: Age, Length: 338, dtype: object